In [7]:
from twarc import Twarc
import pandas as pd
import json

# Initialize twarc with your Twitter API credentials
t = Twarc(consumer_key='QAlOsFqFtWiRVFRndgC31HGBo',consumer_secret='uitKDVrYBbSZZhbJg15xonJShOl0NFLg7iayVKc0aJhCQLtwxl', access_token='1570111679039520773-L2DOWoUADykCMOGNBg1KSYCqsjAYPS', access_token_secret='lJAN4bF31flrPVcY4rlzWf8wTBpw9msrdyXZw85i39HKt')

tweets = []

for line in open('doctors.jsonl', 'r'):
    tweets.append(json.loads(line))

# Create a dataframe
tweets = pd.DataFrame(tweets)
df = pd.DataFrame()

# Extract the set of ids from the json
df["username"] = tweets["includes"].apply(lambda x: x["users"][0]["username"])
df = df.drop_duplicates(subset=['username'])
df["description"] = tweets["includes"].apply(lambda x: x["users"][0]["description"])
df["followers_count"] = tweets["includes"].apply(lambda x: x["users"][0]["public_metrics"]["followers_count"])
df["following_count"] = tweets["includes"].apply(lambda x: x["users"][0]["public_metrics"]["following_count"])

df

,username,description,followers_count,following_count
0,meta201012,International Medical Graduate with a passion ...,14,37
1,skzahr,"MD, PhD | PGY1 Anesthesiology & Pain Medicine ...",333,615
2,AlikaMD,"President, Canadian Medical Association @cma_d...",14024,1662
3,mbcborges07,"M.D. / music lover, #roncaronca addicted!/",167,384
5,GuyMacAdam7,3PPHrWrfFcw1zDj1o1TegSFhHZw1YsAsy4J,0,0
...,...,...,...,...
517,DrTonyEskander,"Husband, Dad, 🇨🇦Head&NeckOnc/Recon @Sunnybrook...",1778,2463
520,SuchitraPai1,"Mom, homemaker,self employed, accountant, fina...",213,520
531,DrCarolynSnider,"Chief of Emergency Medicine, Unity Health Toro...",3699,1458
546,DrDeaRoberts,"MD (retired), BHEc (Nutrition) \nWe shall figh...",1497,1187


In [ ]:
# This cell takes forever run at your own peril

unique_ids = set()

# Iterate through the set of usernames
for index, row in df.iterrows():
    username = row["username"]
    followers_count = row["followers_count"]
    following_count = row["following_count"]
    # check which list is smaller
    if followers_count <= following_count:
        # Retrieve the followers of the user
        followers = list(t.follower_ids(username))
        # add unique ids to the set
        unique_ids.update(followers)
    else:
        # Retrieve the followings of the user
        following = list(t.friend_ids(username))
        # add unique ids to the set
        unique_ids.update(following)
        
# write the set of unique ids to the followers.txt file
with open("followers.txt","w+") as h:
    for id in unique_ids:
        h.write(str(id)+"\n")

In [8]:
import pandas as pd

# Read the CSV file and create a dataframe
df_2 = pd.read_csv('users.csv')


df_2.isnull().sum()


id                                    0
created_at                            0
username                              0
name                                  2
description                        2118
entities.description.cashtags     29943
entities.description.hashtags     22637
entities.description.mentions     24231
entities.description.urls         27714
entities.url.urls                 16601
location                           6626
pinned_tweet_id                   18097
profile_image_url                     2
protected                             0
public_metrics.followers_count        0
public_metrics.following_count        0
public_metrics.listed_count           0
public_metrics.tweet_count            0
url                               16601
verified                              0
verified_type                     29987
withheld.scope                    29986
withheld.copyright                29987
withheld.country_codes            29984
__twarc.retrieved_at                  0


In [9]:
df_2 = df_2[df_2['description'].notna()]

In [10]:
# Create a list of keywords to search for in the bio column
keywords = ['Medical Doctor', 'M.D.', 'MD', 'PhD in Medicine', 'Phd in medicine', 'PHD in Medicine', '#MD', 'Doctor of Medicine', 'Canadian Doctor', 'Dr.',  "Anesthesiology",    "Cardiology",    "Dermatology",    "Emergency Medicine",    "Family Medicine",    "Gastroenterology",    "General Surgery",    "Internal Medicine",    "Neurology",    "Obstetrics and Gynecology",    "Oncology",    "Ophthalmology",    "Orthopedic Surgery",    "Pathology",    "Pediatrics",    "Psychiatry",    "Radiology",    "Urology", "Doctor"]

# Create a list of location keywords
location_keywords = ['Canada', 'canada', "Vancouver", "Calgary", "Edmonton", "Regina", "Saskatoon", "Winnipeg", "Toronto", "Ottawa", "Montreal", "Quebec City", "Halifax", "Victoria", "St. John's", "Charlottetown", "Yellowknife", "Iqaluit", "Whitehorse", "Fredericton", "Kelowna", "Abbotsford", "Barrie", "Hamilton", "Kitchener", "London, Canada", "Windsor", "St. Catharines", "Guelph", "Kingston", "Ottawa-Gatineau", "Moncton", "Sarnia", "Sherbrooke", "Trois-Rivieres", "Thunder Bay", "Prince George", "Sault Ste. Marie", "Red Deer", "Bracebridge", "Nanaimo", "Fort McMurray", "Grande Prairie", "Sudbury", "Saint John", "Brandon", "Penticton", "Kamloops", "Chilliwack", "Prince Rupert", "Fort St. John", "Cranbrook", "Medicine Hat", "Lethbridge", "Timmins", "Dawson Creek", "Kawartha Lakes", "Belleville", "North Bay", "Ajax", "Peterborough", "Woodstock", "Aldershot", "Aurora", "Brantford", "Burlington", "Cambridge", "Clarington", "Cobourg", "Collingwood", "Cornwall", "Dartmouth", "Delta", "Dryden", "Etobicoke", "Fort Erie", "Gatineau", "Guelph", "Halton Hills", "Hamilton", "Kanata", "Kapuskasing", "Keswick", "King City", "Kingston", "Kitchener", "Langley", "Leamington", "Lincoln", "London, Canada", "Markham", "Milton", "Mississauga", "Newmarket", "Niagara Falls", "North York", "Oakville", "Oshawa", "Ottawa", "Owen Sound", "Pickering", "Port Colborne", "Prince Albert", "Red Deer", "Richmond Hill", "Sarnia", "Sault Ste. Marie", "Scarborough", "St. Catharines", "St. Thomas", "Strathroy", "Temiskaming Shores", "Thorold", "Thunder Bay", "Trenton", "Vaughan", "Waterloo", "Welland", "Weston", "Whitby", "Willowdale", "Windsor", "Woodstock", "York, Canada"]

# Use boolean indexing to filter the dataframe based on the keywords and location_keywords
df = df_2[df_2['description'].str.contains('|'.join(keywords), case=False) & df_2['location'].str.contains('|'.join(location_keywords), case=False)]

In [11]:
df.shape

(1068, 27)

In [12]:
df.drop(columns=['entities.description.cashtags',	'entities.description.hashtags',	'entities.description.mentions'	,'entities.description.urls'	,'entities.url.urls'])


df 


,id,created_at,username,name,description,entities.description.cashtags,entities.description.hashtags,entities.description.mentions,entities.description.urls,entities.url.urls,...,public_metrics.tweet_count,url,verified,verified_type,withheld.scope,withheld.copyright,withheld.country_codes,__twarc.retrieved_at,__twarc.url,__twarc.version
60,1236031388933074951,2020-03-06T20:50:30.000Z,ScottZablotny,Scott Zablotny,MS4 @thenosm | Interested in Emergency Medicin...,NaN,NaN,"[""@thenosm"", ""@uOttawa""]",NaN,NaN,...,9,NaN,False,NaN,NaN,NaN,NaN,2023-01-24T13:03:50+00:00,https://api.twitter.com/2/users?tweet.fields=a...,2.13.0
62,56772461,2009-07-14T18:27:34.000Z,ShorensteinCtr,Shorenstein Center,"Shorenstein Center on Media, Politics and Publ...",NaN,NaN,"[""@Kennedy_School"", ""@JournoResource""]",NaN,"[""http://shorensteincenter.org/""]",...,18121,http://shorensteincenter.org/,False,NaN,NaN,NaN,NaN,2023-01-24T13:03:50+00:00,https://api.twitter.com/2/users?tweet.fields=a...,2.13.0
70,3673789718,2015-09-24T20:01:12.000Z,EmergMed558,UGME EM UAlberta,University of Alberta's Emergency Medicine Und...,NaN,NaN,NaN,NaN,NaN,...,438,NaN,False,NaN,NaN,NaN,NaN,2023-01-24T13:03:50+00:00,https://api.twitter.com/2/users?tweet.fields=a...,2.13.0
71,4915464449,2016-02-15T20:17:06.000Z,jashearn,Jason Hearn,IM @ Dal & Medicine @ MUN👨‍⚕️ | Biomedical Eng...,NaN,NaN,NaN,NaN,NaN,...,77,NaN,False,NaN,NaN,NaN,NaN,2023-01-24T13:03:50+00:00,https://api.twitter.com/2/users?tweet.fields=a...,2.13.0
106,765234247141515268,2016-08-15T17:10:36.000Z,GenWSHancock,Winfield Scott Hancock 🇺🇲 🇺🇦 🇹🇼 🟩🦁☀️🟥 ♣️,"Major General, II Corps, #NAFO Army of Potomac...",NaN,"[""#NAFO""]",NaN,NaN,"[""https://gettysburg.stonesentinels.com/union-...",...,5239,https://gettysburg.stonesentinels.com/union-he...,False,NaN,NaN,NaN,NaN,2023-01-24T13:03:51+00:00,https://api.twitter.com/2/users?tweet.fields=a...,2.13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29879,51230826,2009-06-26T21:20:46.000Z,KatieGriffinCTV,Katie Griffin,@CTVOttawa Reporter/Anchor. Instructor. Lover ...,NaN,NaN,"[""@CTVOttawa""]",NaN,"[""http://CTVNewsOttawa.ca""]",...,9598,http://CTVNewsOttawa.ca,False,NaN,NaN,NaN,NaN,2023-01-24T13:08:10+00:00,https://api.twitter.com/2/users?tweet.fields=a...,2.13.0
29898,3497229613,2015-09-08T21:34:13.000Z,mniels4,Michelle Nielsen,"Medical Physicist, transplant recipient, amate...",NaN,NaN,NaN,NaN,NaN,...,90,NaN,False,NaN,NaN,NaN,NaN,2023-01-24T13:08:11+00:00,https://api.twitter.com/2/users?tweet.fields=a...,2.13.0
29899,570148659,2012-05-03T15:38:20.000Z,CThornton32,Christina Thornton,Assistant Prof @caldommed | CFF Postdoc Fellow...,NaN,NaN,"[""@caldommed"", ""@umich"", ""@Ucalgary"", ""@ucalga...",NaN,NaN,...,1358,NaN,False,NaN,NaN,NaN,NaN,2023-01-24T13:08:11+00:00,https://api.twitter.com/2/users?tweet.fields=a...,2.13.0
29922,70486502,2009-08-31T20:07:51.000Z,donnafansk,Donna Fan,| @Usask Medicine Class of '16 | \n\nInterests...,NaN,NaN,"[""@Usask""]",NaN,NaN,...,233,NaN,False,NaN,NaN,NaN,NaN,2023-01-24T13:08:11+00:00,https://api.twitter.com/2/users?tweet.fields=a...,2.13.0


In [13]:
df['username'].to_csv("usernames.txt", index=False, header=False)

In [14]:
import jsonlines

useractivity = []

# Load the first JSON file
with jsonlines.open('question_4.json') as f:
    useractivity.extend(f)

# Load the second JSON file
with jsonlines.open('question_4_2.json') as f:
    useractivity.extend(f)

# Write the result to a new file
with jsonlines.open("useractivity.jsonl", mode='w') as f:
    f.write_all(useractivity)

activity = pd.DataFrame(useractivity)

activity

,data,includes,meta,__twarc,errors
0,"[{'conversation_id': '1305964036593528832', 'l...","{'media': [{'type': 'photo', 'url': 'https://p...","{'result_count': 3, 'newest_id': '130602635889...",{'url': 'https://api.twitter.com/2/users/12360...,NaN
1,"[{'entities': {'mentions': [{'start': 1, 'end'...","{'users': [{'url': 'https://t.co/00oNzNngtQ', ...","{'result_count': 100, 'newest_id': '1311380294...",{'url': 'https://api.twitter.com/2/users/56772...,"[{'resource_id': 'kathytpham', 'parameter': 'e..."
2,[{'text': 'RT @MattPerault: A deep dive on yes...,{'users': [{'created_at': '2009-07-14T18:27:34...,{'next_token': '7140dibdnow9c7btw3t4674yjt2ule...,{'url': 'https://api.twitter.com/2/users/56772...,"[{'value': 'Goodman_Writer', 'detail': 'Could ..."
3,"[{'entities': {'urls': [{'start': 117, 'end': ...","{'users': [{'verified': False, 'username': 'Sh...",{'next_token': '7140dibdnow9c7btw3t3qzgoucqrlw...,{'url': 'https://api.twitter.com/2/users/56772...,"[{'value': 'aprilaser', 'detail': 'Could not f..."
4,"[{'conversation_id': '1263869356351651841', 'l...","{'media': [{'type': 'photo', 'url': 'https://p...",{'previous_token': '77qpymm88g5h9vqkluif7ez1lc...,{'url': 'https://api.twitter.com/2/users/56772...,"[{'value': 'aprilaser', 'detail': 'Could not f..."
...,...,...,...,...,...
2362,"[{'reply_settings': 'everyone', 'author_id': '...","{'users': [{'username': 'mniels4', 'public_met...","{'result_count': 21, 'newest_id': '13096687357...",{'url': 'https://api.twitter.com/2/users/34972...,NaN
2363,[{'edit_history_tweet_ids': ['1310353551907061...,"{'media': [{'type': 'photo', 'url': 'https://p...",{'next_token': '7140dibdnow9c7btw3t3qzew28sx64...,{'url': 'https://api.twitter.com/2/users/57014...,"[{'value': '1268265706220433408', 'detail': 'C..."
2364,"[{'id': '1262246356665552897', 'in_reply_to_us...",{'users': [{'public_metrics': {'followers_coun...,{'previous_token': '77qpymm88g5h9vqkluif7csime...,{'url': 'https://api.twitter.com/2/users/57014...,"[{'value': '1173990600565121024', 'detail': 'C..."
2365,"[{'referenced_tweets': [{'type': 'retweeted', ...",{'users': [{'entities': {'description': {'ment...,{'previous_token': '77qpymm88g5h9vqkluicoaveqc...,{'url': 'https://api.twitter.com/2/users/57014...,NaN


In [15]:
tweets_df_l = []
for dictionary in activity['data'].tolist():
    tweets_df_l.append(pd.DataFrame(dictionary))

In [16]:
tweets_df = pd.concat(tweets_df_l).reset_index(drop=True)
tweets_df

,conversation_id,lang,id,created_at,reply_settings,context_annotations,entities,attachments,edit_controls,in_reply_to_user_id,text,author_id,edit_history_tweet_ids,public_metrics,possibly_sensitive,referenced_tweets,geo,withheld
0,1305964036593528832,en,1306026358896758784,2020-09-16T00:25:23.000Z,everyone,"[{'domain': {'id': '65', 'name': 'Interests an...","{'urls': [{'start': 295, 'end': 318, 'url': 'h...",{'media_keys': ['3_1306026354333319168']},"{'edits_remaining': 5, 'is_edit_eligible': Tru...",165638031,@nolan_madeleine @ddsv3 @thenosm #NOSMRollCall...,1236031388933074951,[1306026358896758784],"{'retweet_count': 2, 'reply_count': 0, 'like_c...",False,"[{'type': 'replied_to', 'id': '130596403659352...",NaN,NaN
1,1305656494256517121,en,1305656494256517121,2020-09-14T23:55:40.000Z,everyone,NaN,"{'mentions': [{'start': 3, 'end': 9, 'username...",NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,RT @ddsv3: TWEETUP @thenosm @_justinam @nolan_...,1236031388933074951,[1305656494256517121],"{'retweet_count': 4, 'reply_count': 0, 'like_c...",False,"[{'type': 'retweeted', 'id': '1305655825449586...",NaN,NaN
2,1288624317379555328,en,1288624317379555328,2020-07-29T23:55:53.000Z,everyone,NaN,"{'urls': [{'start': 186, 'end': 209, 'url': 'h...",NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,Inspiring TEDx Talk given by my mentor @moradh...,1236031388933074951,[1288624317379555328],"{'retweet_count': 1, 'reply_count': 0, 'like_c...",False,NaN,NaN,NaN
3,1311380294671503360,en,1311380294671503360,2020-09-30T19:00:01.000Z,everyone,"[{'domain': {'id': '45', 'name': 'Brand Vertic...","{'mentions': [{'start': 1, 'end': 12, 'usernam...",NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,.@BostonJoan discusses a coordinated push of c...,56772461,[1311380294671503360],"{'retweet_count': 6, 'reply_count': 0, 'like_c...",False,NaN,NaN,NaN
4,1311370559675158529,en,1311370559675158529,2020-09-30T18:21:20.000Z,everyone,"[{'domain': {'id': '131', 'name': 'Unified Twi...","{'mentions': [{'start': 139, 'end': 147, 'user...",NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,"Did you miss our event ""Confronting Disinforma...",56772461,[1311370559675158529],"{'retweet_count': 14, 'reply_count': 0, 'like_...",False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191522,1214995668500135937,en,1214995668500135937,2020-01-08T19:41:55.000Z,everyone,NaN,"{'mentions': [{'start': 3, 'end': 13, 'usernam...",NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,RT @McMasterU: McMaster is deeply saddened by ...,818154242477277186,[1214995668500135937],"{'retweet_count': 141, 'reply_count': 0, 'like...",False,"[{'type': 'retweeted', 'id': '1214989671425609...",NaN,NaN
191523,1197943524332752901,en,1198209259630141440,2019-11-23T11:58:43.000Z,everyone,NaN,"{'mentions': [{'start': 0, 'end': 13, 'usernam...",NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",279368185,@EdsonBellido Thank you Edson!,818154242477277186,[1198209259630141440],"{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,"[{'type': 'replied_to', 'id': '119809670781014...",NaN,NaN
191524,1198034748020592641,en,1198079967260631041,2019-11-23T03:24:58.000Z,everyone,NaN,"{'annotations': [{'start': 29, 'end': 31, 'pro...",NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",1042092036,@xufei0304 Thank you so much Fei! :),818154242477277186,[1198079967260631041],"{'retweet_count': 0, 'reply_count': 0, 'like_c...",False,"[{'type': 'replied_to', 'id': '119803474802059...",NaN,NaN
191525,1197943524332752901,en,1197943524332752901,2019-11-22T18:22:47.000Z,everyone,NaN,"{'annotations': [{'start': 33, 'end': 40, 'pro...",NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,It was truly an honor! Thank you McMaster! htt...,818154242477277186,[1197943524332752901],"{'retweet_count': 1, 'reply_count': 1, 'like_c...",False,"[{'type': 'quoted', 'id': '1197920713333780482'}]",NaN,NaN


In [17]:
tweets_df['created_at'] = pd.to_datetime(tweets_df['created_at'])

In [18]:
def get_top_n(df = tweets_df, regex='(#[^\s]+)', n=10):
    
    return pd.Series(df.text.str.extractall(regex)[0].tolist()).value_counts()[:n]

In [24]:
import emoji

def extract_emojis(s):
    return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

In [25]:
tweets_df_en = tweets_df[tweets_df.lang == 'en']

In [26]:
get_top_n(df=tweets_df_en, regex='(#[^\s]+)', n=10)

#COVID19        5312
#MedEd           672
#COVID19.        653
#cannabis        603
#coronavirus     572
#covid19         499
#mmj             482
#ottnews         447
#MedTwitter      441
#health          409
dtype: int64

In [27]:
get_top_n(df=tweets_df_en, regex='(@[^\s]+)', n=10)

@picardonhealth:    558
@JustinTrudeau      552
@fordnation         517
@UHN                479
@uoftmedicine       436
@ctvottawa          414
@Safimod            409
@CIHR_IRSC          380
@picardonhealth     378
@CFMSFEMC           376
dtype: int64

In [28]:
tweets_df_en['emojis'] = tweets_df_en.text.apply(lambda x: emoji.emoji_list(x))

/var/folders/hj/n94x31sj08bdmgwjs6ybxbf40000gn/T/ipykernel_22593/581539383.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df_en['emojis'] = tweets_df_en.text.apply(lambda x: emoji.emoji_list(x))


In [29]:
emoji_l = []
for e in tweets_df_en['emojis'].tolist():
    if len(e)>0:
        for i in e:
            emoji_l.append(i['emoji'])

In [30]:
pd.Series(emoji_l).value_counts()[:10]

❤️    1923
😂     1885
🇨🇦    1184
👏      908
🙏      700
👇      571
🎉      560
😊      553
😍      468
🤣      466
dtype: int64

In [31]:
tweets_df_en.loc[:, 'retweet_count'] = tweets_df_en.public_metrics.apply(lambda x: x['retweet_count'])

/var/folders/hj/n94x31sj08bdmgwjs6ybxbf40000gn/T/ipykernel_22593/396124695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df_en.loc[:, 'retweet_count'] = tweets_df_en.public_metrics.apply(lambda x: x['retweet_count'])


In [33]:
tweets_df_en['date'] = tweets_df_en.created_at.dt.date

retweet_count_df = tweets_df_en.groupby('date').retweet_count.sum().reset_index()

/var/folders/hj/n94x31sj08bdmgwjs6ybxbf40000gn/T/ipykernel_22593/1761169564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df_en['date'] = tweets_df_en.created_at.dt.date


In [34]:
import plotly.express as px

fig = px.line(retweet_count_df, x="date", y="retweet_count", title='Timeline evolution for the total number of retweets')
fig.show()

In [35]:
pandemic_word_list = ['covid', 'virus', 'pandemic', 'sars', 'covid', 'mask', 'health', 'medic', 'doctor']

In [36]:
tweets_df_en_pandemic = tweets_df_en[tweets_df_en.text.str.contains('|'.join(pandemic_word_list), case=False)]

In [37]:
tweets_df_en_pandemic = tweets_df_en[tweets_df_en.text.str.contains('|'.join(pandemic_word_list), case=False)]

In [38]:
tweet_count_pandemic_df = tweets_df_en_pandemic.groupby('date').author_id.count().reset_index()

## Extra

In [39]:
import re
import pandas as pd
def mentions_counter(text_list):
    #join tweets
    allwords = ' '.join( [tweet for tweet in text_list] ).lower()
    #mention finder
    mentions = re.findall(r'@(\w+)', allwords)
    #count distinct values of the series
    mentions = pd.Series(mentions).value_counts()
    mentions = mentions.to_frame()
    return mentions

mentions_counter(activity.iloc[1,1])

/var/folders/hj/n94x31sj08bdmgwjs6ybxbf40000gn/T/ipykernel_22593/1382736355.py:9: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



,0


In [40]:
allwords = ' '.join(activity.iloc[1,1] ).lower()
allwords

'users tweets media'

In [41]:
new_lst = " ".join(" ".join(map(str, sub)) for sub in activity.data[0])
new_lst

'conversation_id lang id created_at reply_settings context_annotations entities attachments edit_controls in_reply_to_user_id text author_id edit_history_tweet_ids public_metrics possibly_sensitive referenced_tweets conversation_id lang id created_at reply_settings entities edit_controls text author_id edit_history_tweet_ids public_metrics possibly_sensitive referenced_tweets conversation_id lang id created_at reply_settings entities edit_controls text author_id edit_history_tweet_ids public_metrics possibly_sensitive'